In [3]:
!pip install rapidfuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 43.4 MB/s eta 0:00:00


In [2]:
import string
import json
import copy
from tqdm import tqdm
import re
import pandas as pd

In [ ]:
train_df = pd.read_csv("data/seniority_labelled_development_set_cleaned.csv")
test_df = pd.read_csv("data/seniority_labelled_test_set_cleaned.csv")

train_df.iloc[1131]

,1131
job_id,33027740
job_title,Fitter & Welder
job_summary,A food manufacturing company has an opportunit...
job_ad_details,A well known food manufacturer operating since...
classification_name,"Manufacturing, Transport & Logistics"
subclassification_name,Other
y_true,experienced


# RBIC

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen2.5-0.5B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [ ]:
def call_model(messages):
  text = tokenizer.apply_chat_template(
      messages,
      tokenize=False,
      add_generation_prompt=True
  )
  model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

  generated_ids = model.generate(
      **model_inputs,
      max_new_tokens=512
  )
  generated_ids = [
      output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
  ]

  response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
  return response

In [ ]:
def fewshot(messages):
  few_shot_indices = [2081,1,5,12,3]


  for i in few_shot_indices:
      desc = {
        "job_title": train_df.iloc[i].job_title,
        "job_summary": train_df.iloc[i].job_summary,
        "job_ad_details": train_df.iloc[i].job_ad_details,
        "classification_name": train_df.iloc[i].classification_name,
        "subclassification_name": train_df.iloc[i].subclassification_name
      }
      desc_str = str(desc)

      # add the description
      messages.append({
        "role": "user",
        "content": desc_str
      })

      label = train_df.iloc[i].y_true
      label_str = str(label)

      # add the output
      messages.append({
          "role": "assistant",
          "content": label_str
      })

  return messages

In [ ]:
#== step 1: role explanation
def step_1():
  messages = [
        {"role": "system", "content": "You are an expert job ad annotator. Your task is to infer the seniority information from job descriptions. The seniority label may be present in the set: [intermediate, senior, lead, head, experienced, entry-level, executive, assistant, senior/lead, deputy, director, trainee, associate, graduate, junior, general-manager, coordinator, student, chief, principal, apprentice, qualified, entry-level to intermediate, senior associate, standard, senior assistant, specialist, mid-level, entry level assistant, experienced assistant, manager, graduate/junior, independent, 1st year apprentice, senior-executive, junior assistant, assistant manager, supervisor, second-in-command, associate director, board, 4th year apprentice, mid-senior, regional head, middle-management, advanced, 2nd year apprentice, intermediate apprentice, level 2, assistant head, owner, post-doctoral, owner-operator, middle management, senior head, assistant director, junior-intermediate, sous, intermediate to senior, senior executive] . If not present in the set, then create a label."},
        {"role": "user", "content": "Based on your role, can you briefly explain what seniority means, and what seniority labels look like?"}
    ]


  response = call_model(messages)
  messages.append({"role": "assistant", "content": response[0]})
  return response[0], messages


In [ ]:
#== step 2: setting sub-task --> ask for salary patterns
def step_2(messages):
  messages.append({
        "role": "user",
        "content": "As a seniority label predictor, what are some common phrases or patterns that indicate a seniority label in a job description?"
    })

  response = call_model(messages)
  messages.append({"role": "assistant", "content": response[0]})
  return response, messages

In [ ]:
#== step 3: presence of seniority (skipped)
def step_3(messages_static, desc_str):
    messages = copy.deepcopy(messages_static)

    messages.append({
        "role": "user",
        "content": f"{desc_str} does this job description include any seniority-related information? Just respond with 'Yes' or 'No'."
    })

    response = call_model(messages)
    messages.append({"role": "assistant", "content": response[0]})

    p3_content = response[0]

    # clean and check the response
    response_p3 = p3_content.translate(str.maketrans('', '', string.punctuation))
    response_p3 = response_p3.strip().lower()[:3]
    #print(f"step3 messages: {len(messages)}")
    return response_p3, messages

In [ ]:
#== step 4: iterative coaching/finding clues to prevent hallucination
def step_4(response_p3, messages):
    messages.append({
            "role": "user",
            "content": "Extract the seniority-related phrases from the text verbatim. Respond in JSON: {\"Clue\": \"\"}."
        })

    response = call_model(messages)
    messages.append({"role": "assistant", "content": response[0]})
    #print(f"step4 messages: {len(messages)}")

    return response[0], messages

In [ ]:
#== step 5: use the clue to generate the final output
def step_5(messages):
    messages.append({
        "role": "user",
        "content": (
            "Based on the extracted seniority clue, return a structured seniority label in the format {\"seniority_label\": \"seniority label\"}. Do return anything else apart from this. "

        )
    })

    response = call_model(messages)
    answer_str = response[0]
    return answer_str

In [ ]:
def RBIC_static_messages(verbose=False, add_fewshot=True):
    response, messages = step_1()
    if verbose: print(f"RB step 1: {response}\n")

    response, messages = step_2(messages)
    if verbose: print(f"RB step 2: {response}\n")

    if add_fewshot:
        messages = fewshot(messages)
        if verbose: print(f"Fewshot examples added\n")
    return messages


In [ ]:
def RBIC(messages, desc_str, verbose=False):
    response_p3, messages_local = step_3(messages, desc_str)
    if verbose: print(f"IC step 1: {response_p3}\n")

    response, messages_local = step_4(response_p3, messages_local)
    if verbose: print(f"IC step 2: {response}\n")

    answer_str = step_5(messages_local)


    try:
        # extract the value
        data = json.loads(answer_str)
        label = data["seniority_label"]
    except json.JSONDecodeError:
        # Handle the case where the response is not valid JSON
        print(f"Error: Invalid JSON response: {answer_str}")
        label = "ERROR" + str(answer_str)

    if verbose: print(f"IC step 3 (Final): {label}\n")
    if verbose: print(f"IC step 3 (Final Raw): {answer_str}\n")

    return str(label)

# Qualitative Testing with few shot

In [ ]:
messages_static = RBIC_static_messages(verbose=True, add_fewshot=True)

RB step 1: Seniority refers to the duration of time spent within a particular position or department. It indicates how long someone has been employed at that specific position. Seniority labels typically show this duration in years. For example:

- Entry-level to intermediate: This might indicate someone who has been with the company for less than one year but more than two years.
- Intermediate to senior: This might indicate someone who has been there for three years or more.

Please note that these are general examples and the exact meaning may vary depending on the context and industry.

RB step 2: ['Here are some common phrases or patterns that could indicate a seniority label in a job description:\n\n1. "Senior" - Often used to describe a person\'s experience in a leadership capacity.\n2. "Director" - Indicates a higher-level management role.\n3. "Assistant Director" - Describes a middle-level management role.\n4. "Manager" - Typically used when describing a professional position 

In [ ]:
ind = 28

#[2081,1,5,12,3]

desc = {
    "job_title": train_df.iloc[ind].job_title,
    "job_summary": train_df.iloc[ind].job_summary,
    "job_ad_details": train_df.iloc[ind].job_ad_details,
    "classification_name": train_df.iloc[ind].classification_name,
    "subclassification_name": train_df.iloc[ind].subclassification_name,
  }
desc_str = str(desc)

label = train_df.iloc[ind].y_true
label_str = str(label)

print(f"len of messages_static {len(messages_static)}")
label_pred = RBIC(messages_static, desc_str, verbose=True)


print(f"pred = {label_pred}")
print(f"truth = {label_str}")

len of messages_static 15
IC step 1: yes

IC step 2: {"Clue": "Senior"}

IC step 3 (Final): intermediate

IC step 3 (Final Raw): {"seniority_label": "intermediate"}

pred = intermediate
truth = senior


In [ ]:
ind = 395

#[2081,1,5,12,3]

desc = {
    "job_title": test_df.iloc[ind].job_title,
    "job_summary": test_df.iloc[ind].job_summary,
    "job_ad_details": test_df.iloc[ind].job_ad_details,
    "classification_name": test_df.iloc[ind].classification_name,
    "subclassification_name": test_df.iloc[ind].subclassification_name,
  }
desc_str = str(desc)

label = test_df.iloc[ind].y_true
label_str = str(label)

print(f"len of messages_static {len(messages_static)}")
label_pred = RBIC(messages_static, desc_str, verbose=True)


print(f"pred = {label_pred}")
print(f"truth = {label_str}")

len of messages_static 15
IC step 1: yes

IC step 2: ```json
{"Clue": "Seniority-related phrases:"}
```

Error: Invalid JSON response: ```json
{"seniority_label": "highly influential"}
```
IC step 3 (Final): ERROR```json
{"seniority_label": "highly influential"}
```

IC step 3 (Final Raw): ```json
{"seniority_label": "highly influential"}
```

pred = ERROR```json
{"seniority_label": "highly influential"}
```
truth = lead


# Quantative testing

In [ ]:
# df to store model predictions
test_pred_df = pd.DataFrame(columns=["y_pred"])
test_pred_df

,y_pred


In [ ]:
test_pred_df2 = pd.DataFrame(columns=["y_pred"])
test_pred_df2

,y_pred


In [ ]:
range(len(test_df))

range(0, 689)

In [ ]:
range(len(test_df[:200]))

range(0, 200)

In [ ]:
for i in tqdm(range(len(test_df[:200]))):
    print("In step",i)
    desc = {
        "job_title": test_df.iloc[i].job_title,
        "job_summary": test_df.iloc[i].job_summary,
        "job_ad_details": test_df.iloc[i].job_ad_details,
        "classification_name": test_df.iloc[i].classification_name,
        "subclassification_name": test_df.iloc[i].subclassification_name
    }
    desc_str = str(desc)

    label = test_df.iloc[i].y_true
    label_str = str(label)
    print(label)

    label_pred = RBIC(messages_static, desc_str)
    print(label_pred)

    test_pred_df2.loc[len(test_pred_df2)] = label_pred

test_pred_df2.to_csv('seniority_labelled_test_set_qwen_preds_few_shot.csv', index=False)

  0%|          | 0/200 [00:00<?, ?it/s]

In step 0
senior


  0%|          | 1/200 [00:06<20:56,  6.31s/it]

Executive
In step 1
experienced


  1%|          | 2/200 [00:12<21:04,  6.39s/it]

...

entry-level
In step 198
senior


100%|█████████▉| 199/200 [23:50<00:07,  7.81s/it]

design & architecture
In step 199
experienced


100%|██████████| 200/200 [23:55<00:00,  7.18s/it]

experienced


# Testing with no few shot

In [ ]:
messages_static = RBIC_static_messages(verbose=True, add_fewshot=False)

RB step 1: Based on my role, I am a Senior Manager at XYZ Corporation. Therefore, my seniority labels would be "Senior" or "Executive".

RB step 2: ['In the job description provided, there are several phrases or patterns that indicate a seniority label:\n\n- "Senior Executive"\n- "Chief Executive Officer (CEO)"\n- "President"\n- "Director of Operations"\n- "Manager of Sales"\n- "Senior Analyst"\n\nThese phrases suggest that the job involves significant leadership responsibilities, often with substantial authority over multiple departments.']



In [ ]:
test_pred_df = pd.DataFrame(columns=["y_pred"])
test_pred_df

,y_pred


In [ ]:
test_pred_df3 = pd.DataFrame(columns=["y_pred"])
test_pred_df3

,y_pred


In [ ]:
for i in tqdm(range(len(test_df[:200]))):
    print("In step",i)
    desc = {
        "job_title": test_df.iloc[i].job_title,
        "job_summary": test_df.iloc[i].job_summary,
        "job_ad_details": test_df.iloc[i].job_ad_details,
        "classification_name": test_df.iloc[i].classification_name,
        "subclassification_name": test_df.iloc[i].subclassification_name
    }
    desc_str = str(desc)

    label = test_df.iloc[i].y_true
    label_str = str(label)
    print(label)

    label_pred = RBIC(messages_static, desc_str)
    print(label_pred)

    test_pred_df3.loc[len(test_pred_df3)] = label_pred

test_pred_df3.to_csv('seniority_labelled_test_set_qwen_preds_no_few_shot.csv', index=False)

  0%|          | 0/200 [00:00<?, ?it/s]

In step 0
senior


  0%|          | 1/200 [00:03<10:12,  3.08s/it]

Error: Invalid JSON response: ```json
{"seniority_label": "financial planning and modelling manager"}
```
ERROR```json
{"seniority_label": "financial planning and modelling manager"}
```
In step 1
experienced


  1%|          | 2/200 [00:22<42:13, 12.79s/it]

...

Senior Executive
In step 198
senior


100%|█████████▉| 199/200 [10:15<00:03,  3.68s/it]

policy
In step 199
experienced


100%|██████████| 200/200 [10:17<00:00,  3.09s/it]

cleaner


# Regular Prompting

In [ ]:
test_pred_df = pd.DataFrame(columns=["y_pred"])

In [ ]:
messages_static = [
    {"role": "system", "content": "You are an expert job ad annotator. Your task is to infer the seniority information from job descriptions. The seniority label may be present in the set: [intermediate, senior, lead, head, experienced, entry-level, executive, assistant, senior/lead, deputy, director, trainee, associate, graduate, junior, general-manager, coordinator, student, chief, principal, apprentice, qualified, entry-level to intermediate, senior associate, standard, senior assistant, specialist, mid-level, entry level assistant, experienced assistant, manager, graduate/junior, independent, 1st year apprentice, senior-executive, junior assistant, assistant manager, supervisor, second-in-command, associate director, board, 4th year apprentice, mid-senior, regional head, middle-management, advanced, 2nd year apprentice, intermediate apprentice, level 2, assistant head, owner, post-doctoral, owner-operator, middle management, senior head, assistant director, junior-intermediate, sous, intermediate to senior, senior executive] . If not present in the set, then create a label."},
]

In [ ]:
for i in tqdm(range(len(test_df[:200]))):
    desc = {
        "job_title": train_df.iloc[i].job_title,
        "job_summary": train_df.iloc[i].job_summary,
        "job_ad_details": train_df.iloc[i].job_ad_details,
        "classification_name": train_df.iloc[i].classification_name,
        "subclassification_name": train_df.iloc[i].subclassification_name,
    }
    desc_str = str(desc)

    messages = copy.deepcopy(messages_static)
    messages.append({
        "role": "user",
        "content": (
            f"{desc_str} Extract structured salary information from this job descriptions in the format: min-max-currency-frequency. "
            "Respond in JSON: {\"MinSalary\": \"\", \"MaxSalary\": \"\", \"Currency\": \"\", \"Frequency\": \"\"}. "
            "If not provided explicitly, output 0 for \"MinSalary\" and \"MaxSalary\", and \"None\" for \"Currency\" and \"Frequency\". "
            "If the salary is mentioned, always output a range, where MinSalary and MaxSalary can be equal. "
            "Use 'nation_short_desc' to determine the correct currency. "
            "Output the currency as 3 letters. Use adverb to output frequency (annual, monthly, daily or hourly)."
        )
    })

    response = call_model(messages)
    answer_str = response[0]

    # format the output
    try:
        answer_str_ = answer_str[answer_str.find('{'):answer_str.find('}') + 1]
        answer_str_ = answer_str_.replace('“', '"')
        answer_str_ = answer_str_.replace('”', '"')
        answer = json.loads(answer_str_)

        label = f"{answer['MinSalary']}-{answer['MaxSalary']}-{answer['Currency'].upper()}-{answer['Frequency'].upper()}"

    except json.JSONDecodeError:
        print(f"Failed to parse model output as JSON: {answer_str}")
        label = "ERROR " + answer_str

    test_pred_df.loc[len(test_pred_df)] = label

# export the dataframe to a new csv file
test_pred_df.to_csv('seniority_labelled_test_set_qwen_preds_simple_prompt.csv', index=False)

  6%|▋         | 13/200 [00:20<05:18,  1.70s/it]

Failed to parse model output as JSON: To fulfill your request, I need to identify the specific details about the salary structure that was mentioned in the job description. However, since no such information is available, I cannot provide the requested salary ranges. Please provide the actual salary details so I can assist you better.


 40%|████      | 81/200 [01:57<02:43,  1.37s/it]

Failed to parse model output as JSON: {"MinSalary": 92470, "MaxSalary": None, "Currency": "AUD", "Frequency": "Annual"}


 58%|█████▊    | 116/200 [02:49<02:35,  1.85s/it]

...

 64%|██████▍   | 129/200 [03:06<01:27,  1.23s/it]

Failed to parse model output as JSON: [850, 12000, None, Monthly]


 84%|████████▍ | 169/200 [04:05<00:40,  1.30s/it]

Failed to parse model output as JSON: [45, 49999, None, None, Annual]


100%|██████████| 200/200 [04:49<00:00,  1.45s/it]


# Metrics

In [ ]:
few_shot = pd.read_csv("seniority_labelled_test_set_qwen_preds_few_shot.csv")
no_few_shot = pd.read_csv("seniority_labelled_test_set_qwen_preds_no_few_shot.csv")
prompt = pd.read_csv("seniority_labelled_test_set_qwen_preds_simple_prompt.csv")

# Post processing

In [ ]:
few_shot

,y_pred
0,executive
1,entry-level
2,wellbeing administrator
3,executive
4,entry-level
...,...
195,junior
196,entry-level
197,entry-level
198,design & architecture


change label since, lack of escaping causes issues with json load

# **Few_shot**

In [ ]:
few_shot

,y_pred
0,Executive
1,entry-level
2,wellbeing administrator
3,Executive
4,entry-level
...,...
195,junior
196,entry-level
197,entry-level
198,design & architecture


In [ ]:
label = few_shot.iloc[20].y_pred
test =  label.replace("ERROR", "").replace("```json", "").replace("```", "").strip()
print(test)

{"clue": "Seniority related phrase found: \"Senior\"}


In [ ]:
few_shot.iloc[20].y_pred = {
  "seniority_label": "Senior"
}

In [ ]:
import json
json.loads('{"clue": "Seniority related phrase found: \\"Senior\\""}' )

In [ ]:
post_process_fewshot = few_shot

In [ ]:
for i in few_shot.index:
  prediction = few_shot.iloc[i].y_pred

  if isinstance(prediction, str) and "ERROR" in prediction:
    cleaned_text = prediction.replace("ERROR", "").replace("```json", "").replace("```", "").strip()

    try:
      loaded = json.loads(cleaned_text)
      label = f"{loaded['seniority_label'].lower()}"

    except:
      label = None

    post_process_fewshot.loc[i, 'y_pred'] = label


In [ ]:
# make all labels lowercase
post_process_fewshot["y_pred"] = post_process_fewshot["y_pred"].str.lower()

In [9]:
seniority_true = pd.read_csv("seniority_labelled_test_set_cleaned.csv")
seniority_true

,job_id,job_title,job_summary,job_ad_details,classification_name,subclassification_name,y_true
0,38380721,Financial Planning and Modelling Manager,.,Help us create better connected futures.Make ...,Accounting,Financial Accounting & Reporting,senior
1,38960662,Residential Site Manager - Melbourne CBD - Hou...,Housing & Apartment Site Manager for a Tier 1 ...,Residential Site Manager - Melbourne CBD - Re...,Construction,Foreperson/Supervisors,experienced
2,32227649,"Wellbeing Administrator - Fixed Term, part time",Wellbeing Administrator with key focus to ensu...,Hagley College is a designated character state...,Administration & Office Support,Administrative Assistants,entry-level
3,38641145,Senior Natural Hazards and Climate Change Planner,Undertake applied research and associated comm...,Auckland or Lower Hutt GNS Science is New Zeal...,Science & Technology,Other,senior
4,39902251,Credit Union - Member & Customer services,We are looking for a smiley Member Services Of...,"Westforce Credit Union - Smile, We're not a ba...",Call Centre & Customer Service,Customer Service - Call Centre,intermediate
...,...,...,...,...,...,...,...
684,40087169,GP Family Planning,DPV Health GP Superclinic Broadmeadows seeks a...,GP Family Planning Part Time Contract – Split ...,Healthcare & Medical,General Practitioners,experienced
685,38493704,Dispatch Team Member - Kerikeri,Help us pack and load orders so we can deliver...,"Together, let’s build New Zealand! We have a d...","Manufacturing, Transport & Logistics","Warehousing, Storage & Distribution",entry-level
686,35493658,Senior Property Development Officer,Join our Property & Development team to provid...,Dynamic customer centric businessGreat team e...,Engineering,Other,senior
687,39647371,Client Services Associate - Mosman,"Support the operations of our Mosman clinic, c...",About Artisan Aesthetic Group\nArtisan Aesthet...,Trades & Services,Hair & Beauty Services,associate


# Metrics RBIC + fewshot

In [ ]:
import pandas as pd
from rapidfuzz import fuzz

# 2. Combine into one DataFrame
df = pd.DataFrame({
    'y_true': seniority_true[0:200]["y_true"],
    'y_pred': post_process_fewshot["y_pred"]
})

# 3. Exact‐match flag
df['exact_match'] = df['y_true'] == df['y_pred']

# 4. Compute fuzzy‐string similarity (0–100)
df['sim_score'] = df.apply(lambda row: fuzz.ratio(row.y_true, row.y_pred), axis=1)

# 5. Classify similar vs unrelated
THRESHOLD = 70
df['is_similar']   = (~df['exact_match']) & (df['sim_score'] >= THRESHOLD)
df['is_unrelated'] = ~(df['exact_match'] | df['is_similar'])

# 6. Tally
total    = len(df)
exact    = df['exact_match'].sum()
similar  = df['is_similar'].sum()
unrel    = df['is_unrelated'].sum()

print(f"Total examples:       {total}")
print(f"Exact matches:        {exact}  ({exact/total:.1%})")
print(f"Semantically similar: {similar}  ({similar/total:.1%})")
print(f"Unrelated labels:     {unrel}  ({unrel/total:.1%})")

# 7. (Optional) View some borderline cases
print("\n-- Similar but non‐exact (sample):")
print(df[df.is_similar][['y_true','y_pred','sim_score']].head())

print("\n-- Unrelated labels (sample):")
print(df[df.is_unrelated][['y_true','y_pred','sim_score']].head())

Total examples:       200
Exact matches:        21  (10.5%)
Semantically similar: 0  (0.0%)
Unrelated labels:     179  (89.5%)

-- Similar but non‐exact (sample):
Empty DataFrame
Columns: [y_true, y_pred, sim_score]
Index: []

-- Unrelated labels (sample):
         y_true                   y_pred  sim_score
0        senior                executive  26.666667
1   experienced              entry-level  36.363636
2   entry-level  wellbeing administrator  23.529412
3        senior                executive  26.666667
4  intermediate              entry-level  43.478261


# **no_few_shot**

In [ ]:
no_few_shot

,y_pred
0,"ERROR```json\n{""seniority_label"": ""financial p..."
1,Resident
2,wellbeing administrator
3,Senior
4,Member Services Officer
...,...
195,Senior
196,Senior
197,Senior Executive
198,policy


In [ ]:
post_process_no_fewshot = no_few_shot

In [ ]:
for i in no_few_shot.index:
  prediction = no_few_shot.iloc[i].y_pred

  if isinstance(prediction, str) and "ERROR" in prediction:

    cleaned_text = prediction.replace("ERROR", "").replace("```json", "").replace("```", "").strip()

    try:
      loaded = json.loads(cleaned_text)
      label = f"{loaded['seniority_label'].lower()}"
    except:
      label = None

    post_process_no_fewshot.loc[i, 'y_pred'] = label

In [ ]:
post_process_no_fewshot["y_pred"] = post_process_no_fewshot["y_pred"].str.lower()

# Metrics RBIC + no fewshot

In [ ]:
import pandas as pd
from rapidfuzz import fuzz

# 2. Combine into one DataFrame
df = pd.DataFrame({
    'y_true': seniority_true[0:200]["y_true"],
    'y_pred': post_process_no_fewshot["y_pred"]
})

# 3. Exact‐match flag
df['exact_match'] = df['y_true'] == df['y_pred']

# 4. Compute fuzzy‐string similarity (0–100)
df['sim_score'] = df.apply(lambda row: fuzz.ratio(row.y_true, row.y_pred), axis=1)

# 5. Classify similar vs unrelated
THRESHOLD = 70
df['is_similar']   = (~df['exact_match']) & (df['sim_score'] >= THRESHOLD)
df['is_unrelated'] = ~(df['exact_match'] | df['is_similar'])

# 6. Tally
total    = len(df)
exact    = df['exact_match'].sum()
similar  = df['is_similar'].sum()
unrel    = df['is_unrelated'].sum()

print(f"Total examples:       {total}")
print(f"Exact matches:        {exact}  ({exact/total:.1%})")
print(f"Semantically similar: {similar}  ({similar/total:.1%})")
print(f"Unrelated labels:     {unrel}  ({unrel/total:.1%})")

# 7. (Optional) View some borderline cases
print("\n-- Similar but non‐exact (sample):")
print(df[df.is_similar][['y_true','y_pred','sim_score']].head())

print("\n-- Unrelated labels (sample):")
print(df[df.is_unrelated][['y_true','y_pred','sim_score']].head())

Total examples:       200
Exact matches:        22  (11.0%)
Semantically similar: 2  (1.0%)
Unrelated labels:     176  (88.0%)

-- Similar but non‐exact (sample):
        y_true            y_pred  sim_score
127  assistant   sales assistant       75.0
150  assistant  retail assistant       72.0

-- Unrelated labels (sample):
         y_true                                    y_pred  sim_score
0        senior  financial planning and modelling manager  17.391304
1   experienced                                  resident  42.105263
2   entry-level                   wellbeing administrator  23.529412
4  intermediate                   member services officer  28.571429
5   experienced                                    senior  23.529412


# prompting eval

All labels are classified as senior when using regular prompting

In [8]:
prompt = pd.read_csv("seniority_labelled_test_set_qwen_preds_simple_prompt-2.csv")
prompt["y_pred"] =  prompt["y_pred"].str.lower()
prompt["y_pred"].unique()

array(['senior'], dtype=object)

# Metrics prompting

In [11]:
import pandas as pd
from rapidfuzz import fuzz

# 2. Combine into one DataFrame
df = pd.DataFrame({
    'y_true': seniority_true[0:200]["y_true"],
    'y_pred': prompt["y_pred"]
})

# 3. Exact‐match flag
df['exact_match'] = df['y_true'] == df['y_pred']

# 4. Compute fuzzy‐string similarity (0–100)
df['sim_score'] = df.apply(lambda row: fuzz.ratio(row.y_true, row.y_pred), axis=1)

# 5. Classify similar vs unrelated
THRESHOLD = 70
df['is_similar']   = (~df['exact_match']) & (df['sim_score'] >= THRESHOLD)
df['is_unrelated'] = ~(df['exact_match'] | df['is_similar'])

# 6. Tally
total    = len(df)
exact    = df['exact_match'].sum()
similar  = df['is_similar'].sum()
unrel    = df['is_unrelated'].sum()

print(f"Total examples:       {total}")
print(f"Exact matches:        {exact}  ({exact/total:.1%})")
print(f"Semantically similar: {similar}  ({similar/total:.1%})")
print(f"Unrelated labels:     {unrel}  ({unrel/total:.1%})")

# 7. (Optional) View some borderline cases
print("\n-- Similar but non‐exact (sample):")
print(df[df.is_similar][['y_true','y_pred','sim_score']].head())

print("\n-- Unrelated labels (sample):")
print(df[df.is_unrelated][['y_true','y_pred','sim_score']].head())

Total examples:       200
Exact matches:        32  (16.0%)
Semantically similar: 0  (0.0%)
Unrelated labels:     168  (84.0%)

-- Similar but non‐exact (sample):
Empty DataFrame
Columns: [y_true, y_pred, sim_score]
Index: []

-- Unrelated labels (sample):
         y_true  y_pred  sim_score
1   experienced  senior  23.529412
2   entry-level  senior  35.294118
4  intermediate  senior  22.222222
5   experienced  senior  23.529412
6   experienced  senior  23.529412
